# Fresh take

## 1. Initalize sites

In [ ]:
import os
dpth = r'G:\My Drive\FileDump\SCL_Testing\raw'
dpth = r'U:\EC_Backup\SCL_Data\20240912'
dpth = r'E:\GSC_Work\SCL_Data\20240912'


fl = [f for f in os.listdir(dpth) if f.startswith()]
fl.sort()
print(fl)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

f = r"C:\Users\User\GSC_Work\SCL_2024\Flux\eddypro_t_full_output_2025-05-02T224906_exp.csv"
df = pd.read_csv(f,skiprows=[0,2],na_values=-9999)
df.index = pd.to_datetime(df['date'] + ' ' + df['time'])
df['ch4_flux']*=1e3
df.loc[((df['ch4_mean']<1.5)|(df['ch4_mean']>10)),['ch4_flux','ch4_mean']]=np.nan
df.loc[((df['ch4_mean']<df['ch4_mean'].quantile(.01))|(df['ch4_mean']>df['ch4_mean'].quantile(.99))),['ch4_flux','ch4_mean']]=np.nan

df.loc[((df['co2_mole_fraction']<375)|(df['co2_mole_fraction']>600)),['co2_flux','co2_mole_fraction']]=np.nan
df.loc[((df['co2_mole_fraction']<df['co2_mole_fraction'].quantile(.01))|(df['co2_mole_fraction']>df['co2_mole_fraction'].quantile(.99))),['co2_flux','co2_mole_fraction']]=np.nan


df.loc[df['qc_ch4_flux']>0,'ch4_flux']=np.nan
df.loc[df['qc_co2_flux']>0,'co2_flux']=np.nan
df.loc[df['u*']<=0.15,['ch4_flux','co2_flux']]=np.nan
df.loc[df['co2_flux']>=20,['ch4_flux','co2_flux']]=np.nan
df.loc[df['co2_flux']<=-20,['ch4_flux','co2_flux']]=np.nan
for f in ['co2_flux','ch4_flux']:
    df.loc[((df[f]>df[f].quantile(.99))|(df[f]<df[f].quantile(.01))),f]=np.nan

fig,ax=plt.subplots(2)
ax[0].plot(df['co2_flux'])
ax[1].plot(df['co2_mole_fraction'])

# df['u*']
fig.autofmt_xdate()

df[['co2_flux','co2_mole_fraction','ch4_flux','ch4_mean']].describe()

In [ ]:
import matplotlib.pyplot as plt

from parseFiles import parseCSI,baseMethods
import importlib
importlib.reload(parseCSI)
importlib.reload(baseMethods)
import os
TOB3 = r'parseFiles\example_data\Flux_Data640.dat'
outputPath = r'E:\GSC_Work\SCL_Testing\intermediate'
# f = parseCSI.parseTOB3(sourceFile=TOB3,dropCols=['Diagnostic_CSAT'])
# tmp = baseMethods.binBundle(variableMap=f.variableMap,DataFrame=f.DataFrame,filename=f.fileTimestamp,outputPath=outputPath)
t = []
r = []
import time
for f in fl:
    if f.endswith('.dat'):
        t1 = time.time()
        TOB3 = os.path.join(dpth,f)
        f = parseCSI.parseTOB3(sourceFile=TOB3,dropCols=['Diagnostic_CSAT'])
        tmp = baseMethods.binBundle(variableMap=f.variableMap,DataFrame=f.DataFrame,filename=f.fileTimestamp,outputPath=outputPath,verbose=False)
        t.append(f.DataFrame.index[0])
        print((f.fileTimestamp))
        diag_codes = (tmp.DataFrame[['Diagnostic_7700','RSSI']].groupby('Diagnostic_7700').agg(['count','mean']))
        r.append(tmp.DataFrame['RSSI'].median())
        print(time.time()-t1)
plt.figure()
plt.scatter(t,r)

In [ ]:
import numpy as np
import pandas as pd

f = 'G:/My Drive/FileDump/SCL_Testing/intermediate/2024_09_08_0900.ecf32'
# f = r"\\wsl.localhost\Ubuntu\home\jskeeter\FluxCalc_engine\custom\binData\2024_07_28_1930.float32_array"
# print(f)
# float32_array = np.fromfile(f,dtype='float32')
# print(float32_array.shape)
# fn = f'{tmp.filename}.ecf32'
# f = os.path.join(tmp.outputPath,fn)
print(f)
ecf32 = np.fromfile(f,dtype='float32')
ecf32 = ecf32.reshape(-1,36000)

plt.figure()
for i in range(0,4):
    plt.plot(ecf32[i])
# pd.to_datetime(np.fromfile('2024_07_28_1430.POSIX_timestamp'),unit='s')

In [ ]:

fn = f'{tmp.filename}.tsf64'
f = os.path.join(tmp.outputPath,fn)
print(f)
tsf64 = np.fromfile(f,dtype='float64')
print(ecf32.reshape(tsf64.shape[0],-1).shape)
print(float32_array.reshape(tsf64.shape[0],-1).shape)

In [ ]:
from parseFiles import parseCSV
import pandas as pd
sf = r'example_data\20240914\20750528-SHSC.SSM.SGT.240720_240913readout.csv'
import importlib
importlib.reload(parseCSV)
f = parseCSV.hoboCSV(sourceFile=sf,skiprows=1)
print(f.variableMap)
f.DataFrame.head()

In [ ]:

from parseFiles import baseMethods
importlib.reload(baseMethods)
from dataclasses import dataclass, field
import numpy as np
import sys

@dataclass(kw_only=True)
class parseDEF(baseMethods.genericLoggerFile):
    defFile: str
    defTimestamp: str = None
    programName: str = None
    LoggerModel: str = None
    DEF: list = field(default_factory=lambda:[])
    Arrays: dict = field(default_factory=lambda:{})
    # variableMap: dict = field(default_factory=lambda:{})
    
    def __post_init__(self):
        with open(self.defFile,'r',encoding='utf-8-sig') as f:
            self.DEF = f.readlines()
            
        arrID = '-1'
        for i,l in enumerate(self.DEF):
            if i == 0:
                self.defTimestamp  = l.rstrip()
            elif i == 1:
                self.defTimestamp = self.defTimestamp + ' ' + l.rstrip()
                self.defTimestamp = pd.to_datetime(self.defTimestamp,format='%m/%d/%Y %H:%M:%S').strftime(format=self.__dataclass_fields__['fileTimestamp'].default)
            k = 'Program:'
            if k in l:
                self.programName = l.split(k)[-1].rstrip('-\n').lstrip()
            k = 'Wiring for'
            if k in l:
                self.LoggerModel = l.split(k)[-1].rstrip('-\n').lstrip()
            if 'Output_Table' in l:
                l = l.rstrip('-\n').replace('  ',' ').split(' ')
                arrID = l[0]
                frequency = pd.to_timedelta(self.parseFreq(f"{l[2]} {l[3]}")).total_seconds()
                self.Arrays[arrID] = {}
                self.Arrays[arrID]['variableMap'] = {}
                self.Arrays[arrID]['DataFrame'] = []
                self.Arrays[arrID]['Frequency'] = str(frequency)+'s'
            elif arrID != '-1' and l == '\n':
                arrID = '-1'
            elif arrID != '-1' :
                l = l.rstrip('\n').split(' ')
                operation = 'Smp'
                if l[0] == '1':
                    name = 'ArrayID'
                    dataType = 'int32'
                else:
                    name = l[1]
                    if len(name.split('_'))>1:
                        operation = name.split('_')[-1]
                    if operation == 'RTM':
                        dataType = 'int32'
                    else:
                        dataType = 'float32'
                self.Arrays[arrID]['variableMap'][name] = {}
                self.Arrays[arrID]['variableMap'][name]['operation'] = operation
                self.Arrays[arrID]['variableMap'][name]['dataType'] = dataType
                self.Arrays[arrID]['variableMap'][name]['ignore'] = dataType == 'float32'
                
@dataclass(kw_only=True)
class parseMixedArray(parseDEF):
    sourceFile: str
    
    def __post_init__(self):
        with open(self.sourceFile) as file:
            MA = [l.rstrip('\n').split(',') for l in file.readlines()]
        super().__post_init__()
        for i,row in enumerate(MA):
            if len(row) == len(self.Arrays[row[0]]['variableMap']):
                self.Arrays[row[0]]['DataFrame'].append(row[1:])
            else:
                if self.verbose:print(f'row {i} length does not match {self.defFile}')
        for arrID in self.Arrays:
            self.Arrays[arrID]['DataFrame'] = pd.DataFrame(self.Arrays[arrID]['DataFrame'],
                                                           columns=list(self.Arrays[arrID]['variableMap'].keys())[1:])
            na_value=['6999','-6999']
            for val in na_value:
                self.Arrays[arrID]['DataFrame'] = self.Arrays[arrID]['DataFrame'].replace(val,np.nan)
            dtCols = ['Year_RTM', 'Day_RTM', 'Hour_Minute_RTM']
            if not sum([c in self.Arrays[arrID]['DataFrame'] for c in dtCols]) == len(dtCols):
                sys.exit('Timestamp format currently not supported.  Should ba a simple fix')
            Date = self.Arrays[arrID]['DataFrame'][['Year_RTM', 'Day_RTM']].astype(int).astype(str).agg(' '.join,axis=1)
            Date = pd.to_datetime(Date,format = '%Y %j')

            Time = self.Arrays[arrID]['DataFrame']['Hour_Minute_RTM'].astype(int).astype(str)
            Time = [t.zfill(4) for t in Time]
            Time = [t[0:2]+':'+t[2:]+':'+'00' for t in Time]
            Time = pd.to_timedelta(Time)
            self.Arrays[arrID]['DataFrame'].index = Date+Time
            self.Arrays[arrID]['DataFrame'] = self.Arrays[arrID]['DataFrame'].astype('float32')
                
MixedArray = r"U:\EC_Backup\SCL_Data\20250514\WX_data.dat"
DEF_file = r"C:\Users\jskeeter\gsc-permafrost\DataLoggerPrograms\SwissCheeseLake_2024\OverWinter.DEF"
d = parseMixedArray(sourceFile=MixedArray,defFile=DEF_file,verbose=True)
# d.Arrays
d.Arrays['101']['DataFrame'].columns
# df.columns

In [ ]:
import matplotlib.pyplot as plt
df = d.Arrays['101']['DataFrame']#.resample('D').max()
df = df.loc[((df.index.year==2025)&(df.index.month==4))]
fig,ax=plt.subplots(2)
ax[0].scatter(df['Current_AVG'],df['SlrW_AVG'])
ax[1].plot(df['Voltage_AVG'])
# df['Voltage_AVG'].plot(ax=ax[0])
# df['Current_AVG'].plot(ax=ax[1])
# fig.autofmt_xdate()
# df[['Voltage_AVG','Current_AVG']].describe()


In [ ]:
%load_ext autoreload
import os
import os
import shutil
import dbPipeline
import time
from parseFiles.helperFunctions.loadDict import loadDict

In [ ]:
%autoreload 2
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
db = dbPipeline.database(projectPath=projectPath,verbose=False,enableParallel=True)
Sites = r'example_data\projectInventory_Template.yml'
db.projectInventory(newSites=Sites)

template = loadDict(r'example_data\source_Template.yml')
rootPath = r'example_chode'
# for siteID in template:
#     for measurementID in template[siteID]:
#         print(siteID,measurementID)
#         db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])


siteID,measurementID = 'SCL','Met'

template[siteID][measurementID]['rootPath'] = r'C:\Users\jskeeter\gsc-permafrost\SCL_2024\20240912'
db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])

In [ ]:
%autoreload 2

df = dbPipeline.databaseFolder(path=r'C:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Met',Years=[2024])
df.dataOut.columns

In [ ]:
df.dataOut.to_csv(r'C:\Users\jskeeter\OneDrive - NRCan RNCan\Documents\Presentations\CANCH4\Data\SCL_met.csv')

In [ ]:
from dataclasses import dataclass

@dataclass
class test:
    a: str = 'a'

    def __post_init__(self,b=None):
        print(b)

test().__post_init__('x')

In [ ]:
from dataclasses import dataclass,field,asdict

@dataclass
class a:
    a:str=field(default='a',repr=True)
    z:str=field(default='z',repr=False)

    def __post_init__(self):
        self.b = 'b'
    
    def blorb(self):
        self.chou = 'pi'

c = a()
c.blorb()
def filt(dc):
    print(dc)
    return({'a':'a'})
asdict(c,dict_factory=dict)



In [ ]:
asdict

In [ ]:
from parseFiles.helperFunctions.loadDict import loadDict
# importlib.reload(dbPipeline)

# db = dbPipeline.database(projectPath=projectPath,verbose=False,enableParallel=False)
template = loadDict(r'example_data\source_Template.yml')
# # sourcePath = r'U:\EC_Backup\SCL_2024\20240724'
sourcePath = r'example_data'
for siteID in template:
    for measurementID in template[siteID]:
        db.rawFileSearch(siteID,measurementID,sourcePath=sourcePath,**template[siteID][measurementID])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
importlib.reload(dbPipeline)
path = projectPath + r'\database\SCL\Met'
path = projectPath + r'\database\SCL\WSM_Profile'
f = dbPipeline.databaseFolder(path = path,Years= 2024)
fig = plt.figure()
Data = f.dataOut
# Data.loc[((Data['AirTC_Avg']>50)|(Data['AirTC_Avg']<-60)),'AirTC_Avg']=np.nan
# plt.plot(Data['AirTC_Avg'])
plt.plot(Data['TS_1_2_1'])
plt.plot(Data['TS_2_2_1'])
plt.plot(Data['TS_3_2_1'])
plt.plot(Data['TS_4_2_1'])
fig.autofmt_xdate()

In [ ]:
import matplotlib.pyplot as plt
import rawDataFile
import importlib
importlib.reload(rawDataFile)
rd = rawDataFile.TOB3(sourceFile=r'u:\EC_Backup\SCL_2024\20240912\Flux_Data495.dat')
# rd.Data['Diagnostic_CSAT']=0
rd.Data.to_csv('SCL_Data_'+rd.fileTimestamp+'.dat',index=False)
# 'Flux_Data_'+rd.fileTimestamp+'.dat'
# rd.fileTimestamp
# rd.Data.to_csv('Test.dat')
# plt.figure()
# plt.plot(rd.Data['Diagnostic_CSAT'][:-1])
rd.Data.shape


In [ ]:
import configparser

ini = configparser.ConfigParser()
ini.read(r'config_files/GHG_md_template.metadata')
for s in ini.sections():
    if s != 'Files':
        for key,value in (ini[s].items()):
            ini[s][key] = ''
ini

with open(r'config_files/GHG_ep_template.metadata', 'w') as f:
    f.write(';GHG_METADATA\n')
    ini.write(f,space_around_delimiters=False)